In [ ]:
!pip install pdfplumber

In [ ]:
import pdfplumber
import pandas as pd
import re

In [ ]:
def read_format_bank_statement(filename: str) -> pd.DataFrame:
    """read and format bank statement for analysis.

    Args
        filename (str): filename of pdf bank statement
    Returns
        df (pd.DataFrame): formatted pd.DataFrame of transactions.
    """
    def _rename_descriptions(df: pd.DataFrame) -> pd.DataFrame:
        """rename descriptions to make it more readable.

        Args
            df (pd.DataFrame): dataframe of activity from bank statement
        Returns
            df (pd.DataFrame): rename bank activity descriptions
        """
        rename_descriptions = {
            "AMAZON": ["AMZN", "AMAZON"],
            "TESCO": ["TESCO"],
            "ASDA": ["ASDA"],
            "CO-OP": ["CO-OP"],
            "ARGOS": ["ARGOS"],
            "H&M": ["HANDM", "HMHENNE"],
            "LIDL": ["LIDL"],
            "COSTA": ["COSTA"],
            "BOOHOO": ["BOOHOO"],
            "ZARA": ["ZARA"],
            "NEXT": ["NEXT"],
            "NEWLOOK": ["NEWLOOK"],
            "SAINSBURY": ['SAINSBURY'],
            "RANDFARMPARK": ["RANDFARMPARK.C"]
        }

        for shop in rename_descriptions.keys():
            for var in rename_descriptions[shop]:
                df.loc[df['description'].str.startswith(var), "description"] = shop

        return df

    def _read_bank_statement(filename: str) -> pd.DataFrame:
        """read bank statement from pdf.

        Args
            filename (str): filename for bank statement pdf
        Returns
            df (pd.DataFrame): bank transaction activity extracted from pdf
        Todo
            # t = "09JUN 12JUN 17454276 SQ*ICEDREAMING Lincoln £3.50"   
            # match = re.search("^(?P<transaction_date>[0-9]{2}[A-Z]{3}).+([A-Z]+)\s.+?(?P<price>£[0-9\.]+)?$", t)
            # match.group("price")
        """
        pdf = pdfplumber.open(filename)

        results = {
            "transaction_date": [],
            "description": [],
            "price": []
        }

        for page in pdf.pages:
            for text in page.extract_text_lines():
                if re.match("^([0-9]{2}[A-Z]{3}\s){2}[0-9]", text['text']):
                    transaction_date, _, _, description, *_, price =  text['text'].split(" ")
                elif re.match("^([0-9]{2}[A-Z]{3}\s){2}[A-Z]", text['text']):
                    transaction_date, _, description, *_, price =  text['text'].split(" ")
                else:
                    continue
                results['transaction_date'].append(transaction_date)
                results['description'].append(description)
                results['price'].append(price)

        return pd.DataFrame(results)

    def _assign_categories(df: pd.DataFrame) -> pd.DataFrame:
        """map descriptions to categories
        
        Args
            df (pd.DataFrame): pd.DataFrame of transactions
        Returns
            df (pd.DataFrame): pd.DataFrame enriched with categories
        """
        category_mapper = {
            "ASDA": "SUPERMARKET",
            "TESCO": "SUPERMARKET",
            "ALDI": "SUPERMARKET",
            "LIDL": "SUPERMARKET",
            "SAINSBURY": "SUPERMARKET",
            "CO-OP": "SUPERMARKET",
            "AMAZON": "ONLINE SHOPPING",
            "ARGOS": "ONLINE SHOPPING",
            "ASOS": "CLOTHES",
            "NEXT": "CLOTHES",
            "ZARA": "CLOTHES",
            "BOOHOO": "CLOTHES",
            "NEWLOOK": "CLOTHES",
            "H&M": "CLOTHES",
            "COSTA": "EATING OUT",
            "STARBUCKS": "EATING OUT",
            "MCDONALDS": "EATING OUT",
            "THEBOATHOUSETHORPE": "EATING OUT",
            "RSSTHORPEONTHE": "PETROL",
            "WHISBYANIMALPARKLIM": "DAYS OUT",
            "RANDFARMPARK": "DAYS OUT",      
            "THEBOTANISTLINCOLN": "PUB",
            "THESTRAITANDNARROW": "PUB",
            "FROG&FIDDLE": "PUB"
        }

        return df.assign(
            category=df.description.map(category_mapper)
        )
        
    df = _read_bank_statement(filename=filename)

    df = (
        df.assign(
            price=df.price.str.replace("[£,]", "", regex=True),
            description=df.description.str.upper().replace(".COM|WWW.|.CO.UK|HTTPS://", "", regex=True),
            transaction_date=pd.to_datetime(df.transaction_date+"2023", format='%d%b%Y'), # fix year
        )
        .pipe(_rename_descriptions)
        .pipe(_assign_categories)
        .astype(
            {
                "price": "float64",
                "description": "string",
                "transaction_date": "string",
                "category": "string"
            }
        )
        .sort_values(by="transaction_date")
    )

    return df

def category_total(df: pd.DataFrame, category: str) -> float:
    return round(sum(df[df.category == category].price), 2)

def total_incomings(df: pd.DataFrame) -> float:
    incomings = df.price < 0
    return round(sum(df[incomings]['price']), 2)

def total_outgoings(df: pd.DataFrame) -> float:
    outgoings = df.price > 0
    return round(sum(df[outgoings]['price']), 2)

def max_transaction(df: pd.DataFrame) -> pd.DataFrame:
    return df[df.price == max(df.price)]

def mode_description(df: pd.DataFrame) -> pd.DataFrame:
    return df[df.description == df.description.mode()[0]]

def summarise_categories(df: pd.DataFrame) -> pd.DataFrame:
    return df.groupby("category")['price'].agg("sum").reset_index()

In [ ]:
df = read_format_bank_statement("tesco_july.pdf")